In [31]:
import requests
import json
import re
from tqdm import tqdm
import os
import pandas as pd
import time
import uuid

In [2]:
def fetch_save_read(url, file, reader=pd.read_csv, sep='\t', **kwargs):
  ''' Download file from {url}, save it to {file}, and subsequently read it with {reader} using pandas options on {**kwargs}.
  '''
  if not os.path.exists(file):
    if os.path.dirname(file):
      os.makedirs(os.path.dirname(file), exist_ok=True)
    df = reader(url, sep=sep, index_col=None)
    df.to_csv(file, sep=sep, index=False)
  return pd.read_csv(file, sep=sep, **kwargs)

In [3]:
organism = "Mammalia/Homo_sapiens"
url = 'ftp://ftp.ncbi.nih.gov/gene/DATA/GENE_INFO/{}.gene_info.gz'.format(organism)
file = '{}.gene_info.tsv'.format(organism)

ncbi_gene = fetch_save_read(url, file)


In [4]:
def maybe_split(record):
    ''' NCBI Stores Nulls as '-' and lists '|' delimited
    '''
    if record in {'', '-'}:
        return set()
    return set(record.split('|'))

def supplement_dbXref_prefix_omitted(ids):
    ''' NCBI Stores external IDS with Foreign:ID while most datasets just use the ID
    '''
    for id in ids:
        # add original id
        yield id
        # also add id *without* prefix
        if ':' in id:
            yield id.split(':', maxsplit=1)[1]

In [5]:
ncbi_gene['All_synonyms'] = [
    set.union(
      maybe_split(gene_info['Symbol']),
      maybe_split(gene_info['Symbol_from_nomenclature_authority']),
      maybe_split(str(gene_info['GeneID'])),
      maybe_split(gene_info['Synonyms']),
      maybe_split(gene_info['Other_designations']),
      maybe_split(gene_info['LocusTag']),
      set(supplement_dbXref_prefix_omitted(maybe_split(gene_info['dbXrefs']))),
    )
    for _, gene_info in ncbi_gene.iterrows()
  ]

synonyms, gene_id = zip(*{
    (synonym, gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
    for synonym in gene_info['All_synonyms']
  })
ncbi_lookup_syn = pd.Series(gene_id, index=synonyms)
symbols, cap, gene_id = zip(*{
    (gene_info['Symbol'], gene_info['Symbol'].upper(), gene_info['GeneID'])
    for _, gene_info in ncbi_gene.iterrows()
  })
ncbi_lookup_sym = pd.Series(gene_id, index=symbols)
ncbi_lookup_sym_cap = pd.Series(gene_id, index=cap)

In [6]:
index_values = ncbi_lookup_syn.index.value_counts()
ambiguous = index_values[index_values > 1].index
ncbi_lookup_syn_disambiguated = ncbi_lookup_syn[(
(ncbi_lookup_syn.index == ncbi_lookup_syn) | (~ncbi_lookup_syn.index.isin(ambiguous))
)]
sym_dict = ncbi_lookup_sym.to_dict()
syn_dict_cap = ncbi_lookup_sym_cap.to_dict()
syn_dict = ncbi_lookup_syn_disambiguated.to_dict()
def gene_lookup(gene):
    gene_id = sym_dict.get(gene)
    if gene_id: return str(gene_id)
    gene_id = syn_dict_cap.get(gene)
    if gene_id: return str(gene_id)
    return str(syn_dict.get(gene))

In [21]:
gene_lookup('HLA-A')

'3105'

In [26]:
gene_name_mapper = {
    "Seladin": "DHCR24",
 'AA555029_RC': "AA555029_RC",
 'ELOC (TCEB1)': "ELOC",
 'ERBB2 (HER2)': "ERBB2",
 'HLA_A': "HLA-A",
 'ROS1\n': "ROS1",
}


In [27]:
invalid_genes = set()
def valid_gene(gene):
    gene = gene_name_mapper.get(gene, gene)
    if gene_lookup(gene) != 'None':
        return True
    invalid_genes.add(gene)
    return False

In [32]:
gmt = []
with open('data/lara.gmt') as o:
    for line in o:
        label, description, *genes = line.split("\t")
        valid_genes = []
        for gene in genes:
            if valid_gene(gene):
                valid_genes.append(gene)
        if len(valid_genes) >= 5:
            gmt.append(["%s (%s)"%(label, description), '', *valid_genes])


In [33]:
invalid_genes

{'',
 '\n',
 'AA555029_RC',
 'ARHGDB',
 'CHECK1',
 'DHRF',
 'EGFR3',
 'ER',
 'FDF18',
 'FRAG1',
 'FRDX2',
 'FRKCA',
 'GUS',
 'KIAA0999',
 'KIAA1625',
 'KIE20A',
 'KRTS',
 'LOC100131053',
 'LOC100288906',
 'LOC730018',
 'MAP2K',
 'MSI',
 'ORCL6L',
 'P1K3R3',
 'PALB3',
 'PHGCH',
 'PTNP11',
 'RPLPO',
 'SLCA7A5',
 'TMB'}

In [30]:
with open('data/filtered_biomarkers_from_lara.gmt', 'w') as o:
    o.write("\n".join(["\t".join(i) for i in gmt]))
